# Содержание


- [Создание и обучение модели](#3)
    - [Подготовка данных для обучения](#31)
    - [Обучение моделей](#33)
        - [Деревья решений](#334)

# Создание и обучение модели <a id="3"></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv("cleaned_dataset.csv", index_col=0)

In [3]:
df.head()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
0,0,35,170,75,90.0,1.0,1.0,1,1,120,...,126,92,17.1,1,1.0,21,35,40,1,1
1,0,30,180,80,89.0,0.9,1.2,1,1,130,...,148,121,15.8,1,0.9,20,36,27,3,0
2,0,40,165,75,91.0,1.2,1.5,1,1,120,...,74,104,15.8,1,0.9,47,32,68,1,0
3,0,50,175,80,91.0,1.5,1.2,1,1,145,...,104,106,17.6,1,1.1,29,34,18,1,0
4,0,50,165,60,80.0,1.0,1.2,1,1,138,...,117,104,13.8,1,0.8,19,12,25,1,0


На этом шаге происходит обучение модели. Обучение моделей машинного обучения происходит итерационно - пробуются различные модели, перебираются гиперпараметры, сравниваются значения выбранной метрики и выбирается лучшая комбинация.


## Подготовка данных для обучения <a id="31"></a>

Вначале нужно определить, на каких данных будет обучаться модель, а на каких тестироваться. **Традиционный подход** - это разделение исходного набора данных на 3 части (обучение, валидация и тестирование) с пропорции 60/20/20. В данном случае обучающая выборка используется для обучения модели, а валидация и тестирование для получения значения метрики без эффекта переобучения.

Однако существует и другой подход к разбиению данных - разделение на 2 части (обучение и тестирование) по правилу 80-20 (80% тренировочный, 20% тестовый). Зачастую данный метод применяется в тех случаях, когда отсутствует достаточное количество данных как в обучающем, так и в проверочном наборе.  

Перед тем как начать разбивать данные необходимо выделить из исходного набора данных целевую переменную (столбец `DRK_YN`) и сохранить её в отдельную переменную. Ниже приведён код разделения:

In [4]:
x = df.drop(columns=["DRK_YN"], axis=1)
y = df["DRK_YN"]

В ходе выполнения данной работы будет использован подход с разделением исходной выборки на 2 части с пропорцией 80-20, поскольку данный способ является самым популярным способом разбиения данных. Для того, чтобы разбить данные таким образом, существует специальный метод `train_test_split` в библиотеке `scikit-learn`.

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

### Деревья решений <a id="334"></a>

Классификаторы на основе **деревьев принятия решений** являются привлекательными моделями, когда необходима интерпретируемость данных. Как подсказывает само название, можно предположить, что такая модель разбивает данные, задавая последовательность вопросов. Базируясь на признаках в обучающем наборе, модель на основе дерева принятия решений обучается последовательности вопросов, чтобы выводить метки классов для образцов.

Используя алгоритм принятия решения, начинаем с корня дерева и разбиваем данные по признаку, который дает в результате наибольший *прирост информации (Information Gain)*. В рамках итерационного процесса повторяем описанную процедуру разбиения в каждом узле, пока листовые узлы не станут чистыми. Это означает, что все образцы в каждом узле принадлежат тому же самому классу.

На практике результатом может оказаться очень глубокое дерево с многочисленными узлами, что легко способно привести к переобучению. Таким образом, дерево необходимо *подрезать*, устанавливая предел для его максимальной глубины.

Для того, чтобы разделить узлы в самых информативных признаках, нам понадобится определить целевую функцию, которую необходимо оптимизировать посредством алгоритма обучения дерева. В данном случае целевая функция заключается в доведении до максимума прироста информации при каждом разделении, что определяется следующим образом:
$$IG(D_p, f) = I(D_p) - \sum_{j=1}^n \frac{N_j}{N_p}I(D_j)$$
где: $f$ - признак для выполнения разбиения; $D_p$ и $D_j$ - набор данных родительского и $j$-ого дочернего узла; $I$ - мера загрязнённости; $N_p$ - общее количество образцов в родительском узле; $N_j$ - количество образцов в $j$-ом дочернем узле.

Можно заметить, что прирост информации представляет собой просто разность между загрязнённостью родительского узла и суммой загрязнённостей дочерних узлов - чем ниже загрязнённость дочерних узлов, тем выше прирост информации.

Зачастую применяют следующие 3 меры загрязнённости (или критерии разбиения):
1. **Энтропия ($I_H$)**:
$$I_H(t) = - \sum_{i=1}^c p(i|t) \ \log_2 p(i|t)$$
В данном случае $p(i|t)$ - доля образцов, которые принадлежат классу $i$ для индивидуального узла $t$. </br>
Следовательно, энтропия равна $0$, если все образцы в узле принадлежат тому же самому классу, и максимальна при равномерном распределении классов. Таким образом, можно говорить, что критерий энтропии стремится довести до максимума полное количество информации в дереве.

2. **Критерий Джинни ($I_G$)**:

Загрязнённость Джинни можно понимать как критерий для минимизации вероятности неправильной классификации:
$$I_G(t) = \sum_{i=1}^c p(i|t)(1-p(i|t)) = 1 - \sum_{i=1}^c p(i|t)^2$$ 
Подобно энтропии критерий Джинни максимален, когда классы в полной мере перемешаны, например, в окружении с двоичным классами. Тем не менее, на практике загрязненность Джинни и энтропия выдают очень похожие результаты, и часто не стоит тратить много времени на оценку деревьев с использованием различных критериев загрязнённости вместо экспериментирования с разными параметрами отсечения при подрезке.

3. **Ошибка классификации ($I_{\Epsilon}$)**:

$$I_{\Epsilon} = 1 - max(p(i|t))$$
Ошибка классификации - полезный критерий для подрезки, но не рекомендуется для создания дерева принятия решений, т.к. она менее чувствительна к изменениям в вероятностях классов узлов.

Далее, попробуем построить несколько моделей дерева решений и найдем оптимальную через кросс-валидацию. Перед началом выполнения поставленной задачи - импортируем все необходимые модули:

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Categorical, Real, Integer
import chime
%load_ext chime 

Далее, также как и в предыдущий раз, создадим конвейер из двух составляющих: стандартизатора и самого классификатора.

In [7]:
# Создаем стандартизатор
standardizer = StandardScaler()

# Создаем классификатор
tree = DecisionTreeClassifier(min_samples_leaf=10)

# Создаём конвейер
pipe = Pipeline([("standardizer", standardizer), ("tree", tree)])

Как и в предыдущие разы, определим гиперпараметры, от которых зависит классификатор и которые будут перебираться в процессе кросс-валидации:  

In [8]:
params = {
    "tree__criterion": Categorical(["gini", "entropy", "log_loss"]),
    "tree__max_depth": Integer(10, 20),
    "tree__max_features": Integer(1, len(x_train.columns)),
    "tree__min_samples_leaf": Integer(20, 50),
}

Запустим процесс кросс-валидации и выведем наилучшую комбинацию гиперпараметров для данного классификатора:

In [9]:
%%time
%%chime
tree_classifier = BayesSearchCV(
    estimator=pipe, 
    search_spaces=params,
    cv=5, verbose=1, 
    n_jobs=-3, 
    scoring="roc_auc").fit(
    x_train, y_train
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

После завершения подбора параметров посмотрим на наилучшую их комбинацию, а также наилучшую оценку, полученную на тренировочных данных:

In [10]:
print(
    f"Оптимальные параметры: {tree_classifier.best_params_}\nНаилучшая оценка: {tree_classifier.best_score_}"
)

Оптимальные параметры: OrderedDict([('tree__criterion', 'log_loss'), ('tree__max_depth', 10), ('tree__max_features', 23), ('tree__min_samples_leaf', 50)])
Наилучшая оценка: 0.8059523169995811


Оптимальные параметры: OrderedDict([('tree__criterion', 'log_loss'), ('tree__max_depth', 10), ('tree__max_features', 23), ('tree__min_samples_leaf', 50)])
Наилучшая оценка: 0.8059523169995811